In [ ]:
pip install transformers

     |████████████████████████████████| 3.1 MB 9.4 MB/s 
     |████████████████████████████████| 3.3 MB 37.6 MB/s 
     |████████████████████████████████| 596 kB 37.3 MB/s 
     |████████████████████████████████| 895 kB 32.3 MB/s 
     |████████████████████████████████| 61 kB 524 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# 1. 모델 로드

In [ ]:
from transformers import TFBertForNextSentencePrediction

In [ ]:
model = TFBertForNextSentencePrediction.from_pretrained('klue/bert-base', from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForNextSentencePrediction: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForNextSentencePrediction were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForNextSentencePrediction for predictions without further training.


# 2. 토크나이저 로드

In [ ]:
from transformers import AutoTokenizer

In [ ]:
import tensorflow as tf

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

# 3. 테스트

인덱스 0 : 실제 다음 문장  
인덱스 1 : 서로 상관없는 문장

In [ ]:
prompt = "2002년 월드컵 축구대회는 일본과 공동으로 개최되었던 세계적인 큰 잔치입니다."
next_sentence = "여행을 가보니 한국의 2002년 월드컵 축구대회의 준비는 완벽했습니다."
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

In [ ]:
logits

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 6.8429694, -2.2522385]], dtype=float32)>

In [ ]:
softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print(probs)

tf.Tensor([[9.9988782e-01 1.1218959e-04]], shape=(1, 2), dtype=float32)


In [ ]:
tf.math.argmax(probs, axis=-1).numpy()

array([0])

In [ ]:
prompt = "2002년 월드컵 축구대회는 일본과 공동으로 개최되었던 세계적인 큰 잔치입니다."
next_sentence = "극장가서 로맨스 영화를 보고싶어요"
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

In [ ]:
logits

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-2.4330912,  5.521658 ]], dtype=float32)>

In [ ]:
softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print(probs)

tf.Tensor([[3.5086818e-04 9.9964917e-01]], shape=(1, 2), dtype=float32)


In [ ]:
tf.math.argmax(probs, axis=-1).numpy()

array([1])

# 4. 데이터 로드

In [ ]:
import random
import pandas as pd
import urllib.request
from sklearn.model_selection import train_test_split

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")

('ChatBotData.csv', <http.client.HTTPMessage at 0x7f96478c5850>)

In [ ]:
chatbot_data = pd.read_csv('ChatBotData.csv')

In [ ]:
chatbot_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [ ]:
del chatbot_data['label']

In [ ]:
chatbot_data.head()

,Q,A
0,12시 땡!,하루가 또 가네요.
1,1지망 학교 떨어졌어,위로해 드립니다.
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.
4,PPL 심하네,눈살이 찌푸려지죠.


In [ ]:
chatbot_data.Q

0                         12시 땡!
1                    1지망 학교 떨어졌어
2                   3박4일 놀러가고 싶다
3                3박4일 정도 놀러가고 싶다
4                        PPL 심하네
                  ...           
11818             훔쳐보는 것도 눈치 보임.
11819             훔쳐보는 것도 눈치 보임.
11820                흑기사 해주는 짝남.
11821    힘든 연애 좋은 연애라는게 무슨 차이일까?
11822                 힘들어서 결혼할까봐
Name: Q, Length: 11823, dtype: object

In [ ]:
chatbot_data.A

0                      하루가 또 가네요.
1                       위로해 드립니다.
2                     여행은 언제나 좋죠.
3                     여행은 언제나 좋죠.
4                      눈살이 찌푸려지죠.
                   ...           
11818          티가 나니까 눈치가 보이는 거죠!
11819               훔쳐보는 거 티나나봐요.
11820                      설렜겠어요.
11821    잘 헤어질 수 있는 사이 여부인 거 같아요.
11822          도피성 결혼은 하지 않길 바라요.
Name: A, Length: 11823, dtype: object

In [ ]:
train_data, test_data = train_test_split(chatbot_data, test_size=0.2,shuffle=Tru random_state=777)

In [ ]:
train_data

,Q,A
11703,친구로 좋아하는 건지 이성으로 좋아하는 건지 모르겠어요.,이전과 달라진 점을 느끼지 못했나봐요.
10797,오늘로 짝남 보내기로 했어요.,힘든 결정이었을텐데 맘고생 많았어요.
1699,모기가 너무 많아.,조심하세요.
4536,출근하기 싫어,직장 스트레스가 심한가봐요.
4476,첫 데이트에 뭐하지,야경이 멋져요. 야경구경 가세요.
...,...,...
11468,짝녀 관련돼서 악몽을 자주 꾸는데 불안함.,악몽이라니 걱정되겠어요.
7221,완전 헤어짐 극복 방법 있을까,바쁘게 살거나 취미생활을 해서 다른 생각을 못하게 하는 것 추천해드려요.
1318,돈이라도 많으면 좋을텐데,돈이 전부는 아니지만 돈보다 좋은건 없다고 했어요.
8915,강아지 좋아하는 여자 어때?,마음이 따뜻할 것 같아요.


In [ ]:
test_data

,Q,A
11761,하루종일 썸남 생각만 해. 괜찮을까?,그것 또한 감정의 일부니까요.
924,냉장고에 먹을 게 하나도 없네,슈퍼라도 가서 쇼핑하고 오세요.
3763,인사드리러 갔는데 파혼하는게 나을것 같아,이혼이 아니라 다행입니다.
341,금값 어때,비싸요.
10736,연애상담하더니 둘이 사귀더라,대화를 하다가 친해졌나봐요.
...,...,...
2890,아이돌 오빠들 좋아,삶의 낙이죠.
1763,무서워요,어흥!! 호랑이보다 무섭나요?
1566,매운 닭발 먹고 싶다.,스트레스 받으시는 일 있으신거죠.
4666,친구에게 물어 봐야겠다,친구도 잘 몰라요.


In [ ]:
train_data = train_data.values
test_data = test_data.values

In [ ]:
train_texts = []
train_pairs = []
train_labels = []
for item in train_data:
    train_texts.append(item[0])
    label = random.randint(0, 1)
    train_labels.append(label)
    if label == 0:
        train_pairs.append(item[1])
    else:
        fake = random.choice(train_data)
        train_pairs.append(fake[1])

In [ ]:
test_texts = []
test_pairs = []
test_labels = []
for item in test_data:
    test_texts.append(item[0])
    label = random.randint(0, 1)
    test_labels.append(label)
    if label == 0:
        test_pairs.append(item[1])
    else:
        fake = random.choice(test_data)
        test_pairs.append(fake[1])

In [ ]:
train_texts[:5]

['친구로 좋아하는 건지 이성으로 좋아하는 건지 모르겠어요.',
 '오늘로 짝남 보내기로 했어요.',
 '모기가 너무 많아.',
 '출근하기 싫어',
 '첫 데이트에 뭐하지']

In [ ]:
train_pairs[:5]

['이전과 달라진 점을 느끼지 못했나봐요.',
 '힘든 결정이었을텐데 맘고생 많았어요.',
 '조심하세요.',
 '각자의 프라이버시니까 이해해주세요.',
 '로맨틱해요.']

In [ ]:
train_labels[:5]

[0, 0, 0, 1, 1]

In [ ]:
train_encodings = tokenizer(train_texts, train_pairs, padding=True, truncation=True)
test_encodings = tokenizer(test_texts, test_pairs, padding=True, truncation=True)

In [ ]:
len(train_encodings['input_ids'])

9458

In [ ]:
print(train_encodings['input_ids'][0])

[2, 3949, 2200, 6992, 2259, 7791, 5606, 6233, 6992, 2259, 7791, 3926, 2918, 10283, 18, 3, 4176, 2145, 10338, 1540, 2069, 4491, 2118, 4047, 2075, 2998, 2182, 18, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
print(train_encodings['token_type_ids'][0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
print(train_encodings['attention_mask'][0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
import tensorflow as tf

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
))

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])

학습하기 전 성능

In [ ]:
model.evaluate(test_dataset.batch(32), batch_size=32)

74/74 [==============================] - 13s 86ms/step - loss: 0.8276 - accuracy: 0.6736


[0.8276231288909912, 0.6735729575157166]

학습

In [ ]:
model.fit(train_dataset.batch(32), epochs=1, batch_size=32)

296/296 [==============================] - 84s 217ms/step - loss: 0.4319 - accuracy: 0.7877


학습 후 성능

In [ ]:
model.evaluate(test_dataset.batch(32), batch_size=32)

74/74 [==============================] - 6s 87ms/step - loss: 0.3856 - accuracy: 0.8152


[0.38563284277915955, 0.8152219653129578]

예측

In [ ]:
pred_dataset = tf.data.Dataset.from_tensor_slices(dict(test_encodings))

In [ ]:
prediction = model.predict(pred_dataset.batch(32))

In [ ]:
tf.math.argmax(prediction[0], axis=-1).numpy()

array([1, 0, 1, ..., 1, 0, 1])